In [3]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import plotly.express as px
import plotly.graph_objects as go
import datetime
import os

# Connect to Google Sheets
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('aquilacommercialsheets-923494a59a4b.json', scope)
client = gspread.authorize(credentials)

# Function to fetch fresh data
def get_fresh_data():
    # Open the spreadsheet
    spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1bzpRnUrpBH6l_zX7DtTypczZf5bpYVwqPUG3tzg2vec/edit?gid=1530779476#gid=1530779476'
    spreadsheet_id = spreadsheet_url.split('/')[5]
    sheet = client.open_by_key(spreadsheet_id).get_worksheet(0)
    df = pd.DataFrame(sheet.get_all_records())
    # Convert date columns to datetime with mixed format parsing
    df['DATE OF REQUIREMENT'] = pd.to_datetime(df['DATE OF REQUIREMENT'], format='mixed')
    df['DATE UPDATED'] = pd.to_datetime(df['DATE UPDATED'], format='mixed')

    # Use DATE UPDATED if it exists, otherwise use DATE OF REQUIREMENT
    df['EFFECTIVE_DATE'] = df['DATE UPDATED'].fillna(df['DATE OF REQUIREMENT'])
    # Convert string values to numeric, removing commas and rounding
    df['REQUIRED SF (LOW)'] = pd.to_numeric(df['REQUIRED SF (LOW)'], errors='coerce')
    df['REQUIRED SF (HIGH)'] = pd.to_numeric(df['REQUIRED SF (HIGH)'], errors='coerce')
    df['REQUIRED SF (AVG)'] = (df['REQUIRED SF (LOW)'] + df['REQUIRED SF (HIGH)']) / 2
    
    return df

# Colors, similar to Dash version
COLORS = {
    'background': '#FFFFFF',
    'text': '#2C3E50', 
    'blue': '#00008B',
    'orange': '#DAA520',
    'light_gray': '#F8F9F9'
}

# Get data and process monthly aggregation
df = get_fresh_data()
monthly_data = df.groupby(pd.Grouper(key='EFFECTIVE_DATE', freq='ME')).agg({
    'REQUIRED SF (LOW)': ['sum', 'count'],
    'REQUIRED SF (HIGH)': 'sum',
    'REQUIRED SF (AVG)': ['mean', 'median']
}).reset_index()

# Flatten multi-level columns
monthly_data.columns = [
    'EFFECTIVE_DATE', 'REQUIRED SF (LOW)', 'REQUIRED SF (COUNT)', 
    'REQUIRED SF (HIGH)', 'REQUIRED SF (AVG)', 'REQUIRED SF (MEDIAN)'
]

# --- Figure 1: Sum of requirements (Line for LOW, HIGH) ---
fig1 = px.line(
    monthly_data,
    x='EFFECTIVE_DATE',
    y=['REQUIRED SF (LOW)', 'REQUIRED SF (HIGH)'],
    title='Monthly Total Square Footage Requirements',
    labels={'value': 'Square Footage', 'EFFECTIVE_DATE': 'Date'},
    color_discrete_sequence=[COLORS['orange'], COLORS['blue']]
)
fig1.update_layout(
    plot_bgcolor=COLORS['background'],
    paper_bgcolor=COLORS['background'],
    font=dict(family='Segoe UI', size=12),
    title={'font': dict(family='Segoe UI', size=24)},
    xaxis=dict(
        gridcolor=COLORS['light_gray'],
        tickformat='%b %Y',
        showgrid=True,
        dtick="M1"
    ),
    yaxis=dict(gridcolor=COLORS['light_gray']),
    margin=dict(t=100, b=50, l=50, r=50)
)

# --- Figure 2: Mean & Median of required SF (Line), and COUNT (Bar, secondary y-axis) ---
fig2 = go.Figure()

# Lines for Mean and Median
fig2.add_trace(go.Scatter(
    x=monthly_data['EFFECTIVE_DATE'],
    y=monthly_data['REQUIRED SF (AVG)'],
    mode='lines+markers',
    name='Avg SF (Mean)', 
    line=dict(color=COLORS['orange'])
))
fig2.add_trace(go.Scatter(
    x=monthly_data['EFFECTIVE_DATE'],
    y=monthly_data['REQUIRED SF (MEDIAN)'],
    mode='lines+markers',
    name='Avg SF (Median)',
    line=dict(color=COLORS['blue'])
))

# Bars for COUNT (secondary y-axis)
fig2.add_trace(go.Bar(
    x=monthly_data['EFFECTIVE_DATE'],
    y=monthly_data['REQUIRED SF (COUNT)'],
    name='Record Count',
    yaxis='y2',
    marker_color=COLORS['text'],
    opacity=0.3
))

fig2.update_layout(
    title={
        'text': 'Monthly Average Square Footage Metrics',
        'font': dict(family='Segoe UI', size=24)
    },
    plot_bgcolor=COLORS['background'],
    paper_bgcolor=COLORS['background'],
    font=dict(family='Segoe UI', size=12),
    xaxis=dict(
        title='Date',
        gridcolor=COLORS['light_gray'],
        tickformat='%b %Y',
        dtick="M1",
        showgrid=True
    ),
    yaxis=dict(
        title="Square Footage",
        gridcolor=COLORS['light_gray']
    ),
    yaxis2=dict(
        title="Count",
        overlaying='y',
        side='right',
        titlefont=dict(family='Segoe UI', size=12, color=COLORS['text']),
        tickfont=dict(family='Segoe UI', size=12, color=COLORS['text'])
    ),
    legend=dict(orientation="h", y=-0.2),
    margin=dict(t=100, b=50, l=50, r=50)
)

# --- Save to HTML files in "charts" directory ---
os.makedirs("charts", exist_ok=True)
fig1.write_html("charts/requirements_sf_total.html")
fig2.write_html("charts/requirements_sf_avg.html")

print("Saved charts/requirements_sf_total.html and charts/requirements_sf_avg.html")

Saved charts/requirements_sf_total.html and charts/requirements_sf_avg.html


In [5]:
import aquila_graphing_tools
aquila_graphing_tools.commit_and_push_all("Readme update")